# gdown

In [ ]:
!pip install gdown
!pip install ipython-autotime

In [ ]:
%load_ext autotime

time: 88.3 µs (started: 2021-02-11 08:26:50 +00:00)


In [ ]:
!gdown "https://drive.google.com/uc?id=1Qbi954Bwx-PplM8F_7TrB_blcqcB-bF2"

Downloading...
From: https://drive.google.com/uc?id=1Qbi954Bwx-PplM8F_7TrB_blcqcB-bF2
To: /content/train.csv
100% 305k/305k [00:00<00:00, 93.6MB/s]
time: 1.22 s (started: 2021-02-11 08:26:50 +00:00)


In [ ]:
!gdown "https://drive.google.com/uc?id=1j7h-SUy2gMJBds6eQalw4CFZC3O7Pr05"

Downloading...
From: https://drive.google.com/uc?id=1j7h-SUy2gMJBds6eQalw4CFZC3O7Pr05
To: /content/collaboration_network.edgelist
38.8MB [00:00, 139MB/s] 
time: 2.33 s (started: 2021-02-11 08:26:52 +00:00)


In [ ]:
!gdown "https://drive.google.com/uc?id=1AEn8P631y-pq6szgAbTm538oM6waf-5E"

Downloading...
From: https://drive.google.com/uc?id=1AEn8P631y-pq6szgAbTm538oM6waf-5E
To: /content/author_embedding.csv
683MB [00:03, 203MB/s]
time: 7.52 s (started: 2021-02-11 08:26:54 +00:00)


In [ ]:
!gdown "https://drive.google.com/uc?id=1AEn8P631y-pq6szgAbTm538oM6waf-5E"

Downloading...
From: https://drive.google.com/uc?id=1AEn8P631y-pq6szgAbTm538oM6waf-5E
To: /content/author_embedding.csv
683MB [00:05, 125MB/s] 
time: 18.6 s (started: 2021-02-11 08:27:02 +00:00)


In [ ]:
!gdown "https://drive.google.com/uc?id=1IATZNB9SCDWkPhU1okCKwf_NMFSC_LSn"

Downloading...
From: https://drive.google.com/uc?id=1IATZNB9SCDWkPhU1okCKwf_NMFSC_LSn
To: /content/collaboration_network_weighted.edgelist
71.3MB [00:02, 28.3MB/s]
time: 4.74 s (started: 2021-02-11 08:27:20 +00:00)


# GAE 

In [ ]:
"""
Deep Learning on Graphs - ALTEGRAD - Dec 2020
"""

import scipy.sparse as sp
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder


def normalize_adjacency(A):
    A_tilde = A + sp.identity(A.shape[0])
    D_tilde = sp.diags(A_tilde.sum(axis=1).A1)
    D_pow_tilde = D_tilde.power(-0.5)
    A_normalized = D_pow_tilde @ A_tilde @ D_pow_tilde
    return A_normalized


def sparse_to_torch_sparse(M):
    """Converts a sparse SciPy matrix to a sparse PyTorch tensor"""
    M = M.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((M.row, M.col)).astype(np.int64))
    values = torch.from_numpy(M.data)
    shape = torch.Size(M.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def loss_function(z, adj, device):
    mse_loss = nn.MSELoss()
    
    indices = adj._indices()

    y_pred = list()
    y = list()

    y_pred.append(torch.sum(torch.mul(z[indices[0,:],:], z[indices[1,:],:]), dim=1))
    y.append(torch.ones(indices.size(1)).to(device))

    random_indices = torch.randint(z.size(0), indices.size())
    y_pred.append(torch.sum(torch.mul(z[random_indices[0,:],:], z[random_indices[1,:],:]), dim=1))
    y.append(torch.zeros(random_indices.size(1)).to(device))


    y_pred = torch.cat(y_pred, dim=0)
    y = torch.cat(y, dim=0)
    
    loss = mse_loss(y_pred, y)
    return loss

time: 3.28 s (started: 2021-02-11 08:27:25 +00:00)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GAE(nn.Module):
    """GAE model"""
    def __init__(self, n_feat, dropout , n_hidden_1 = 128 , n_out = 32):
        super(GAE, self).__init__()

        self.fc3 = nn.Linear(n_feat, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, n_out)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()

    def forward(self, x_in, adj):
    
        ##################
        x =  self.fc3 (x_in)
        x = torch.mm(adj, x)
        x = self.tanh(x)
        x = self.dropout(x)
        #-----------.
        x =  self.fc4(x)
        x = torch.mm(adj, x)
        x = self.tanh(x)
        x = self.dropout(x)
        #-----------.
        z =  self.fc5 (x)
        z = torch.mm(adj, z)
        ##################

        return z

time: 31.4 ms (started: 2021-02-11 08:27:28 +00:00)


#   adj & features

In [ ]:
import numpy as np
import networkx as nx
import time
import torch
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.metrics import mean_absolute_error
from tqdm.notebook import tqdm

time: 664 ms (started: 2021-02-11 08:27:28 +00:00)


In [ ]:
# Initialize device
device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

# Hyperparameters
epochs = 2000
learning_rate = 0.001
dropout_rate = 0.2
n_feat = 300

time: 77.9 ms (started: 2021-02-11 08:27:29 +00:00)


In [ ]:
# load the graph
G = nx.read_weighted_edgelist('collaboration_network_weighted.edgelist',
                     delimiter=' ', nodetype=int)
adj = nx.adjacency_matrix(G)  # Obtains the adjacency matrix
adj_norm = normalize_adjacency(adj)  # Normalizes the adjacency matrix
adj_norm.shape

time: 25.8 s (started: 2021-02-11 08:27:29 +00:00)


In [ ]:
n = G.number_of_nodes()
features = np.random.randn(n, n_feat)

features = torch.FloatTensor(features).to(device)
adj_norm = sparse_to_torch_sparse(adj_norm).to(device)

time: 2.96 s (started: 2021-02-11 08:28:12 +00:00)


# loop

In [ ]:
model = GAE(features.shape[1], dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# Train model
t_total = time.time()
for epoch in tqdm(range(epochs)):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    z = model(features, adj_norm)
    loss = loss_function(z, adj_norm, device)
    loss.backward()
    optimizer.step()
    if epoch % 200 == 0:
       print('Epoch: {:04d}'.format(epoch),
              'loss_train: {:.4f}'.format(loss.item()),
              'time: {:.4f}s'.format(time.time() - t))
    if epoch % 1000 == 0 and epoch != 0:
      df_z = pd.DataFrame(z.detach().cpu().numpy(), index = list(G.nodes))
      df_z.index.name = 'authorID'
      df_z.columns = [str(col) + '_GAE' for col in df_z.columns]
      df_z.to_csv('Z_GAE_32_tanh_Ep'+str(epoch)+'_'+str(round(loss.item(),4))+'.csv')
      print('###Z_GAE_32_tanh_Ep'+str(epoch)+'_'+str(round(loss.item(),4))+'.csv###')

df_z = pd.DataFrame(z.detach().cpu().numpy(), index = list(G.nodes))
df_z.index.name = 'authorID'
df_z.columns = [str(col) + '_GAE' for col in df_z.columns]
df_z.to_csv('Z_GAE_32_tanh_Ep'+str(epochs)+'_'+str(round(loss.item(),4))+'.csv')
print('###Z_GAE_32_tanh_Ep'+str(epochs)+'_'+str(round(loss.item(),4))+'.csv###')

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
print()

In [ ]:
#save final file 

z_detach = z.detach().cpu().numpy()
df_z = pd.DataFrame(z_detach, index = list(G.nodes))
df_z.index.name = 'authorID'
df_z.columns = [str(col) + '_GAE' for col in df_z.columns]
df_z.to_csv('Z_GAE_32_tanh_Plusepoch_.csv')

time: 310 ms (started: 2021-02-10 02:50:35 +00:00)
